#### Custom Data Types

You can also define your own custom data types. There are several ways to achieve it.

##### Classes with `__get_validators__`

You use a custom class with a classmethod `__get_validators__`. It will be called to get validators to parse and validate the input data.

> These validators have the same semantics as in `Validators`, you can declare a parameter `config`, `field`, etc.

In [7]:
import re
from pydantic import BaseModel, ValidationError

In [2]:
# https://en.wikipedia.org/wiki/Postcodes_in_the_United_Kingdom#Validation
post_code_regex = re.compile(
    r"(?:"
    r"([A-Z]{1,2}[0-9][A-Z0-9]?|ASCN|STHL|TDCU|BBND|[BFS]IQQ|PCRN|TKCA) ?"
    r"([0-9][A-Z]{2})|"
    r"(BFPO) ?([0-9]{1,4})|"
    r"(KY[0-9]|MSR|VG|AI)[ -]?[0-9]{4}|"
    r"([A-Z]{2}) ?([0-9]{2})|"
    r"(GE) ?(CX)|"
    r"(GIR) ?(0A{2})|"
    r"(SAN) ?(TA1)"
    r")"
)

In [3]:
class PostCode(str):
    """
    Partial UK postcode validation. Note: this is just an example, and is not
    intended for use in production; in particular this does NOT guarantee
    a postcode exists, just that it has a valid format.
    """
    
    @classmethod
    def __get_validators__(cls):
        # one or more validators may be yielded which will be called in the
        # order to validate the input, each validator will receive as an input
        # the value returned from the previous validator
        yield cls.validate
    
    @classmethod
    def validate(cls, v):
        if not isinstance(v, str):
            raise TypeError("string required")
        m = post_code_regex.fullmatch(v.upper())
        if not m:
            raise ValueError("invalid postcode format")
        # you could also return a string here which would mean model.post_code
        # would be a string, pydantic won't care but you could end up with some
        # confusion since the value's type won't match the type annotation
        # exactly
        return cls(f"{m.group(1)} {m.group(2)}")
    
    def __repr__(self):
        return f"PostCode({super().__repr__()})"

In [4]:
class Model(BaseModel):
    post_code: PostCode

In [5]:
model = Model(post_code="sw8 5el")
print(f"{model = }")
print(f"{model.post_code = }")

model = Model(post_code=PostCode('SW8 5EL'))
model.post_code = PostCode('SW8 5EL')


In [6]:
print(f"{Model.schema() = }")

Model.schema() = {'title': 'Model', 'type': 'object', 'properties': {'post_code': {'title': 'Post Code', 'type': 'string'}}, 'required': ['post_code']}


Similar validation could be achieved using `constr(regex=...)` except the value won't be formatted with a space, the schema would just include the full pattern and the returned value would be a vanilla string.

##### Arbitrary Types Allowed

You can allow arbitrary types using the `arbitrary_types_allowed` config in the `Model Config`.

In [8]:
class Pet:
    def __init__(self, name: str):
        self.name = name

In [9]:
class PetOwner(BaseModel):
    pet: Pet
    owner: str
    
    class Config:
        arbitrary_types_allowed = True

In [10]:
pet = Pet(name="Hedwig")
m = PetOwner(owner="Harry", pet=pet)
print(f"{m = }")
print(f"{m.pet = }")
print(f"{m.pet.name = }")
print(f"{type(m.pet) = }")

m = PetOwner(pet=<__main__.Pet object at 0x00000212F85C9310>, owner='Harry')
m.pet = <__main__.Pet object at 0x00000212F85C9310>
m.pet.name = 'Hedwig'
type(m.pet) = <class '__main__.Pet'>


In [11]:
try:
    m = PetOwner(owner="Harry", pet="Hedwig")
    print(m)
except ValidationError as e:
    print(e)

1 validation error for PetOwner
pet
  instance of Pet expected (type=type_error.arbitrary_type; expected_arbitrary_type=Pet)


In [13]:
pet = Pet(name=42)
m = PetOwner(owner="Harry", pet=pet)
print(f"{m = }")
print(f"{m.pet = }")
print(f"{m.pet.name = }")
print(f"{type(m.pet) = }")

m = PetOwner(pet=<__main__.Pet object at 0x00000212F83242E0>, owner='Harry')
m.pet = <__main__.Pet object at 0x00000212F83242E0>
m.pet.name = 42
type(m.pet) = <class '__main__.Pet'>
